In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import autokeras as ak
from numpy import random
import time

In [2]:
#Plot's Making  Packages
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, BoundaryNorm
from matplotlib.collections import LineCollection
from matplotlib import cm

%config InlineBackend. figure_format = 'retina'

In [3]:
model_theta23 = tf.keras.models.load_model('/work/ML4NO/ML/Regression/models_all/0804_theta23_1.h5')
model_theta23.summary()
model_delta = tf.keras.models.load_model('/work/ML4NO/ML/Regression/models_all/0804_delta1.h5')
model_delta.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 244)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 244)               0         
_________________________________________________________________
normalization (Normalization (None, 244)               489       
_________________________________________________________________
dense (Dense)                (None, 1024)              250880    
_________________________________________________________________
re_lu (ReLU)                 (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                32800     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)               

In [4]:
cut_index = np.load('/work/NIO_largerthan1000_index.npy')
data_IO = np.load('/work/NuFit_IO.npz')
data_NO = np.load('/work/NuFit_NO.npz')

In [9]:
input_state = 2
generate = 10

if input_state == 0:
    data_IO_mid = data_IO['ve_dune'][0]
    data_NO_mid = data_NO['ve_dune'][0]
elif input_state == 1:
    data_IO_mid = np.column_stack([data_IO['ve_dune'], data_IO['vu_dune'], data_IO['vebar_dune'], data_IO['vubar_dune']])[0]
    data_NO_mid = np.column_stack([data_NO['ve_dune'], data_NO['vu_dune'], data_NO['vebar_dune'], data_NO['vubar_dune']])[0]
elif input_state == 2:
    data_IO_mid = np.column_stack([data_IO['ve_dune'], data_IO['vu_dune'], data_IO['vebar_dune'], data_IO['vubar_dune'], data_IO['ve_t2hk'], data_IO['vu_t2hk'], data_IO['vebar_t2hk'], data_IO['vubar_t2hk']])[0]
    data_NO_mid = np.column_stack([data_NO['ve_dune'], data_NO['vu_dune'], data_NO['vebar_dune'], data_NO['vubar_dune'], data_NO['ve_t2hk'], data_NO['vu_t2hk'], data_NO['vebar_t2hk'], data_NO['vubar_t2hk']])[0]

data_IO_mid = data_IO_mid[cut_index]
data_NO_mid = data_NO_mid[cut_index]

In [11]:
%%time
N = 1000000
statistics_state = 2

if statistics_state == 0:
    std = 0.01
    data_create_IO = random.normal(data_IO_mid, std*data_IO_mid, size = (N, len(data_IO_mid)))
    data_create_NO = random.normal(data_NO_mid, std*data_NO_mid, size = (N, len(data_IO_mid)))
elif statistics_state == 1:
    data_create_IO = random.poisson(data_IO_mid, size = (N, len(data_IO_mid)))
    data_create_NO = random.poisson(data_NO_mid, size = (N, len(data_NO_mid)))
elif statistics_state == 2:
    std = 0.01
    data_create_IO = random.normal(data_IO_mid, np.sqrt(data_IO_mid), size = (N, len(data_IO_mid)))
    data_create_NO = random.normal(data_NO_mid, np.sqrt(data_NO_mid), size = (N, len(data_IO_mid)))

CPU times: user 2.15 s, sys: 68.1 ms, total: 2.22 s
Wall time: 2.22 s
